In [12]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
X_APP_TOKEN = os.environ.get("X-App-Token")


In [11]:
url = "https://data.cityofchicago.org/resource/x2n5-8w5q.json?$limit=5000"

headers = {
    "Accept": "application/json",
    "X-App-Token": X_APP_TOKEN
}

response = requests.get(url=url, headers=headers)

if response.status_code == 200:
    response_data = response.json()
else:
    print(response.status_code)

df = pd.json_normalize(response_data)

5000

In [20]:
df.sort_values(by='date_of_occurrence', ascending=False)

,case_,date_of_occurrence,block,_iucr,_primary_decsription,_secondary_description,_location_description,arrest,domestic,beat,...,longitude,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu,:@computed_region_rpca_8um6,location.latitude,location.longitude,location.human_address
4999,JG113053,2023-01-11T14:00:00.000,011XX W Hubbard St,0820,THEFT,$500 AND UNDER,RESIDENCE,N,N,1214,...,-87.656551077,41,22620,25,522,46,49,41.889998232,-87.656551077,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
4998,JG112768,2023-01-11T14:00:00.000,001XX W RANDOLPH ST,0870,THEFT,POCKET-PICKING,MOVIE HOUSE / THEATER,N,N,122,...,-87.632801633,22,14310,38,92,36,41,41.88447161,-87.632801633,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
4997,JG112347,2023-01-11T14:00:00.000,012XX S ASHLAND AVE,0860,THEFT,RETAIL THEFT,CONVENIENCE STORE,Y,N,1233,...,-87.666379171,48,14920,29,96,23,43,41.866331725,-87.666379171,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
4996,JG112412,2023-01-11T14:00:00.000,077XX S SOUTH SHORE DR,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,N,N,421,...,-87.552911904,43,22538,39,421,37,24,41.756512772,-87.552911904,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
4995,JG112335,2023-01-11T14:00:00.000,024XX W TOUHY AVE,0820,THEFT,$500 AND UNDER,OTHER (SPECIFY),Y,N,2411,...,-87.691483053,42,22528,20,42,27,12,42.012253154,-87.691483053,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,JG105765,2023-01-04T05:30:00.000,082XX S CAMPBELL AVE,0810,THEFT,OVER $500,STREET,N,N,835,...,-87.685192083,6,4300,69,243,30,8,41.743577935,-87.685192083,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
7,JG156878,2023-01-04T05:30:00.000,044XX W ALTGELD ST,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,APARTMENT,N,Y,2524,...,-87.737775815,7,22615,21,439,17,2,41.926340063,-87.737775815,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2,JG103740,2023-01-04T05:27:00.000,103XX S SACRAMENTO AVE,0820,THEFT,$500 AND UNDER,STREET,N,N,2211,...,-87.6966837,33,4447,73,256,42,33,41.705320489,-87.6966837,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
1,JG103752,2023-01-04T05:12:00.000,046XX N PULASKI RD,2020,NARCOTICS,POSSESS - AMPHETAMINES,POLICE FACILITY / VEHICLE PARKING LOT,Y,N,1722,...,-87.727964573,28,21869,14,257,38,48,41.96593721,-87.727964573,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
